# AI画像検査システム - Colab UI検証
インタラクティブなWebベースUI

## 1. 環境セットアップ

In [ ]:
# 必要なライブラリのインストール
!pip install torch torchvision opencv-python-headless pillow numpy ipywidgets -q
print("パッケージインストール完了")

In [ ]:
# インポート
import torch
import torch.nn as nn
import torchvision.transforms as T
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import io
import base64

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. モデル定義

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel,self).__init__() 
        self.Encoder = nn.Sequential(self.create_convblock(3,16),
                                     nn.MaxPool2d((2,2)),
                                     self.create_convblock(16,32),
                                     nn.MaxPool2d((2,2)),
                                     self.create_convblock(32,64),
                                     nn.MaxPool2d((2,2)),
                                     self.create_convblock(64,128),
                                     nn.MaxPool2d((2,2)),
                                     self.create_convblock(128,256),
                                     nn.MaxPool2d((2,2)),
                                     self.create_convblock(256,512),
                                    )
        self.Decoder = nn.Sequential(self.create_deconvblock(512,256),
                                     self.create_convblock(256,256),
                                     self.create_deconvblock(256,128),
                                     self.create_convblock(128,128),
                                     self.create_deconvblock(128,64),
                                     self.create_convblock(64,64),
                                     self.create_deconvblock(64,32),
                                     self.create_convblock(32,32),
                                     self.create_deconvblock(32,16),
                                     self.create_convblock(16,16),
                                    )
        self.last_layer = nn.Conv2d(16,3,1,1)
                                        
    def create_convblock(self,i_fn,o_fn):
        conv_block = nn.Sequential(nn.Conv2d(i_fn,o_fn,3,1,1),
                                   nn.BatchNorm2d(o_fn),
                                   nn.ReLU(),
                                   nn.Conv2d(o_fn,o_fn,3,1,1),
                                   nn.BatchNorm2d(o_fn),
                                   nn.ReLU()
                                  )
        return conv_block
    
    def create_deconvblock(self,i_fn , o_fn):
        deconv_block = nn.Sequential(nn.ConvTranspose2d(i_fn, o_fn, kernel_size=2, stride=2),
                                      nn.BatchNorm2d(o_fn),
                                      nn.ReLU(),
                                     )
        return deconv_block

    def forward(self,x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        x = self.last_layer(x)           
        return x

print("モデル定義完了")

## 3. モデル初期化

In [ ]:
# グローバル変数
model = CustomModel()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

preprocess = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
])

current_image = None
model_loaded = False

total_params = sum(p.numel() for p in model.parameters())
print(f"モデル初期化完了")
print(f"パラメータ数: {total_params:,}")
print(f"使用デバイス: {device}")

## 4. インタラクティブUI

In [ ]:
# UI コンポーネント
output_area = widgets.Output()

# ボタン
load_model_btn = widgets.Button(description="📁 モデルをロード", button_style='info')
load_image_btn = widgets.Button(description="🖼️ 画像をアップロード", button_style='primary')
sample_btn = widgets.Button(description="🎲 サンプル画像", button_style='warning')
inference_btn = widgets.Button(description="🔍 推論実行", button_style='success')

# しきい値スライダー
threshold_slider = widgets.FloatSlider(
    value=0.01,
    min=0.001,
    max=0.1,
    step=0.001,
    description='しきい値:',
    readout_format='.4f'
)

# ステータス表示
status_label = widgets.HTML(value="<b>ステータス:</b> モデル未ロード")

# レイアウト
control_box = widgets.HBox([load_model_btn, load_image_btn, sample_btn, inference_btn])
param_box = widgets.HBox([threshold_slider])
ui = widgets.VBox([status_label, control_box, param_box, output_area])

display(ui)

In [ ]:
# モデルロード機能
def load_model_callback(b):
    global model, model_loaded
    
    with output_area:
        clear_output(wait=True)
        print("model.pthファイルをアップロードしてください...")
        
        uploaded = files.upload()
        
        if 'model.pth' in uploaded:
            try:
                # バイナリデータからモデルをロード
                checkpoint = torch.load(io.BytesIO(uploaded['model.pth']), map_location=device)
                model.load_state_dict(checkpoint)
                model.eval()
                model_loaded = True
                
                status_label.value = "<b>ステータス:</b> <span style='color:green'>モデルロード済み</span>"
                print("✅ モデルをロードしました")
            except Exception as e:
                print(f"❌ モデルロードエラー: {e}")
        else:
            print("⚠️ model.pthがアップロードされませんでした")

load_model_btn.on_click(load_model_callback)

In [ ]:
# 画像アップロード機能
def load_image_callback(b):
    global current_image
    
    with output_area:
        clear_output(wait=True)
        print("画像ファイルをアップロードしてください...")
        
        uploaded = files.upload()
        
        for filename, data in uploaded.items():
            try:
                current_image = Image.open(io.BytesIO(data)).convert('RGB')
                
                # 画像表示
                plt.figure(figsize=(6, 6))
                plt.imshow(current_image)
                plt.title(f"アップロード画像: {filename}")
                plt.axis('off')
                plt.show()
                
                print(f"✅ 画像をロードしました: {filename}")
                print(f"サイズ: {current_image.size}")
                break
            except Exception as e:
                print(f"❌ 画像ロードエラー: {e}")

load_image_btn.on_click(load_image_callback)

In [ ]:
# サンプル画像生成
def sample_callback(b):
    global current_image
    
    with output_area:
        clear_output(wait=True)
        
        # ランダムなサンプル画像を生成
        np.random.seed(42)  # 再現性のため
        sample_type = np.random.choice(['normal', 'anomaly'])
        
        if sample_type == 'normal':
            # 正常画像（一様パターン）
            img_array = np.ones((256, 256, 3), dtype=np.uint8) * 128
            img_array += np.random.normal(0, 10, (256, 256, 3)).astype(np.uint8)
            title = "サンプル画像（正常パターン）"
        else:
            # 異常画像（ランダムノイズ）
            img_array = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)
            title = "サンプル画像（異常パターン）"
        
        current_image = Image.fromarray(img_array)
        
        # 画像表示
        plt.figure(figsize=(6, 6))
        plt.imshow(current_image)
        plt.title(title)
        plt.axis('off')
        plt.show()
        
        print(f"✅ {title}を生成しました")

sample_btn.on_click(sample_callback)

In [ ]:
# 推論実行機能
def inference_callback(b):
    global current_image, model
    
    with output_area:
        clear_output(wait=True)
        
        if current_image is None:
            print("⚠️ 画像を先にロードしてください")
            return
        
        try:
            print("🔍 推論実行中...")
            
            # 前処理
            input_tensor = preprocess(current_image).unsqueeze(0).to(device)
            
            # 推論
            with torch.no_grad():
                output = model(input_tensor)
            
            # 後処理
            input_np = input_tensor.cpu().squeeze(0).permute(1, 2, 0).numpy()
            output_np = output.cpu().squeeze(0).permute(1, 2, 0).numpy()
            
            input_np = np.clip(input_np, 0, 1)
            output_np = np.clip(output_np, 0, 1)
            
            # 差分計算
            diff = np.abs(input_np - output_np)
            mse = np.mean((input_np - output_np) ** 2)
            
            # 異常判定
            threshold = threshold_slider.value
            is_anomaly = mse > threshold
            
            # 結果表示
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            axes[0].imshow(input_np)
            axes[0].set_title("入力画像")
            axes[0].axis('off')
            
            axes[1].imshow(output_np)
            axes[1].set_title("復元画像")
            axes[1].axis('off')
            
            axes[2].imshow(diff)
            axes[2].set_title("差分画像")
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # 結果サマリー
            print("="*50)
            print("📊 推論結果")
            print("="*50)
            print(f"復元誤差 (MSE): {mse:.6f}")
            print(f"判定しきい値: {threshold:.6f}")
            print(f"最大差分: {np.max(diff):.6f}")
            print(f"平均差分: {np.mean(diff):.6f}")
            print(f"")
            
            if is_anomaly:
                print("🚨 判定: 異常検出")
                print(f"   復元誤差が閾値 {threshold:.6f} を上回りました")
            else:
                print("✅ 判定: 正常")
                print(f"   復元誤差が閾値 {threshold:.6f} 以下です")
            
            if not model_loaded:
                print("")
                print("⚠️ 注意: 学習前のモデルを使用しています")
                print("   実際の異常検知性能は期待できません")
            
        except Exception as e:
            print(f"❌ 推論エラー: {e}")

inference_btn.on_click(inference_callback)

## 5. 使用方法

1. **モデルをロード** (オプション): 学習済みmodel.pthをアップロード
2. **画像をアップロード** または **サンプル画像** で検証用画像を準備
3. **しきい値を調整** (必要に応じて)
4. **推論実行** で異常検知を実行

### 特徴
- 📱 インタラクティブなWebベースUI
- 🎯 リアルタイム結果表示
- 📊 視覚的な比較（入力・復元・差分）
- ⚙️ しきい値の動的調整
- 🎲 サンプル画像での動作確認